In [8]:
from jupyter_imports import *
from copy import deepcopy

In [9]:
# run only once
from utils.MyLogger import ExecutionLogger # use 'prod' for remote after cache_tmp
start = datetime(2022,7,10,0)#,tzinfo=timezone.utc)
#dirname = os.path.join(os.sep, 'tmp','prod','tradeexecutor')#,'prod')
ExecutionLogger.batch_summarize_exec_logs(dirname=dirname,start=start,add_history_context=False,rebuild=True)

'moved 0 logs to unreadable'

In [10]:
files = ['all_by_clientOrderId','all_by_coin','all_by_symbol','all_parameters','all_inventory_manager','all_data','all_risk_recon']
original = {file:pd.read_csv(os.path.join(os.sep, dirname,f'{file}.csv'),parse_dates=['log_time'],index_col=0) for file in files}

ValueError: Missing column provided to 'parse_dates': 'log_time'

In [ ]:
cmap = cm.get_cmap('Spectral')
#fig, ax = plt.subplots(1)

## parameters for reference

In [ ]:
all_parameters = original['all_parameters']
all_parameters['hash'] = all_parameters['log_time'].apply(lambda x: pd.util.hash_pandas_object(all_parameters.groupby('log_time').get_group(x).astype(str)).sum())
all_parameters['param_index'] = all_parameters['hash'].apply(lambda h: list(all_parameters['hash'].unique()).index(h))
all_parameters.drop(columns=['hash'],inplace=True)
param_map = all_parameters.set_index('log_time')['param_index'].to_dict()
param_map

In [ ]:
indexed = deepcopy(original)
for df in indexed.values():
    df['param_index'] = df['log_time'].apply(lambda x:param_map[x])
#colors = matplotlib.cm.rainbow(np.linspace(0, 1, len(all_parameters['param_index'].unique())))

### slippage, fees in bps. filled in USD

In [ ]:
indexed['all_data'].columns

In [ ]:
by_symbol = indexed['all_by_symbol']
by_symbol = by_symbol[by_symbol['log_time']>start.replace(tzinfo=timezone.utc)]
by_symbol['absFilled'] = by_symbol['filledUSD'].apply(np.abs)
by_symbol['total_bps'] = by_symbol['slippage_bps'] + by_symbol['fee']
by_symbol['fee_USD'] = by_symbol['fee']*by_symbol['absFilled']/10000 # in bps for by_symbol, in USD for by_clientorderid :(
by_symbol['slippage_USD'] = by_symbol['slippage_bps']*by_symbol['absFilled']/10000
by_symbol['total_USD'] = by_symbol['total_bps']*by_symbol['absFilled']/10000

In [ ]:
df_symbol = by_symbol[by_symbol['index']=='average'].pivot_table(index='log_time',aggfunc=sum)
df_symbol[['fee_USD','slippage_USD','total_USD']] /= 10000*by_symbol[by_symbol['index']=='average'].pivot_table(index='log_time',values=['absFilled'],aggfunc=sum).values

In [ ]:
#ax.plot(df, c=df['param_index'], s=120, cmap=cmap, edgecolor='None',figsize=(20,10))
#plt.show()
df_symbol[['fee_USD','slippage_USD','total_USD','absFilled']].plot.bar(secondary_y='absFilled',figsize=(20,5))

### slippage distribution

In [ ]:
print(by_symbol[['total_bps','fee']].describe())
by_symbol.loc[by_symbol['index']=='average','total_bps'].hist(figsize=(10,5))

In [ ]:
by_coin = indexed['all_by_coin']
trackrecord = by_coin[by_coin['index']=='average'].pivot_table(index='log_time',values=['perleg_fee_bps','perleg_filled_USD','premium_vs_inception_bps'],aggfunc=np.mean)
print(trackrecord['premium_vs_inception_bps'].describe())
trackrecord.plot.bar(secondary_y='perleg_filled_USD',figsize=(20,10))

# tick to ack

In [ ]:
by_clientOrderId = indexed['all_by_clientOrderId']
tick_to_order_local = by_clientOrderId[['pending_to_ack_local']]
tick_to_order_local.hist(bins=50,figsize=(10,5))
tick_to_order_local.describe()

# remote vs local (~ round trip)

In [ ]:
round_trip = pd.DataFrame(columns=['round_trip'], data = 2*(indexed['all_data']['timestamp']-indexed['all_data']['remote_timestamp']))
round_trip.hist(bins=50,figsize=(20,10))
round_trip.describe()